In [ ]:
#default_exp load_config

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#export
import os
import json
import jsonschema
import errno

from typing import Dict, Any
from dataclasses import dataclass
from jsonschema import validate
from pathlib import Path

# The `load_config` module

> This module contains the tools necessary to load a `json` configuration file (`config.json`). 

This module provides the tools to open and load sc2trainingg configuration file. The `config.json` file contains the information other modules such as `ingest` use to locate the replays that training grounds will process, the binary for StarCraft II, and the configuration data for the MongoDB database.

Additionally, the module uses `jsonchema` to validate that the `config.json` file contains the proper fields and that the values of those fields are of the appropriate types. 

The information from the file is stored in a `Config_settings` object.

## The `Config_settings` class

In [ ]:
#export
# Configuration of a frozen dataclass for the json file content 
# to assure inmutability. 
@dataclass(frozen=True)
class Config_settings:
    """
    
    ---
    
    Frozen `dataclass` that describes the configuration attributes of SC2 Training Grounds
    
    ---

    **Attributes:**
        *`port_adress (str)`:* address for the server running the MongoDB service
        *`port_number (int)`:* port number that indicates the proper service for the MongoDB service in the server.
        *`db_name (str)`:* name of the database that needs to be accessed
        *`replay_path (str)`:* path to the folder containing the SC2 replays to process. Use absolute path to prevent problems.
    """
    port_address: str
    port_number: int
    db_name: str
    replay_path: str

In [ ]:
show_doc(Config_settings, title_level=3)

<h3 id="Config_settings" class="doc_header"><code>class</code> <code>Config_settings</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>Config_settings</code>(**`port_address`**:`str`, **`port_number`**:`int`, **`db_name`**:`str`, **`replay_path`**:`str`)

Frozen `dataclass` that describes the configuration attributes of SC2 Training Grounds

---

**Attributes:**
    *`port_adress (str)`:* address for the server running the MongoDB service
    *`port_number (int)`:* port number that indicates the proper service for the MongoDB service in the server.
    *`db_name (str)`:* name of the database that needs to be accessed
    *`replay_path (str)`:* path to the folder containing the SC2 replays to process. Use absolute path to prevent problems.

In [ ]:
#exporti
Config_schema = {
    "type": "object",
    "properties":{
        "DB_NAME": {"type":"string"},
        "STEP_MULT": {"type":"number"},
        "MATCH_UPS":  {"type":"array"},
        "SC2_PATH":  {"type":"string"},
        "PORT_ADDRESS":  {"type":"string"},
        "PORT_NUMBER": {"type":"number"},
        "REPLAY_PATH": {"type":"string"}
    }
}

def validate_config_file(file: Path, schema: Dict[str, Any]) -> bool:
    """Review if a given `file` Path fits a predefined `jsonschema`

    Args: 
        file (Path): the path to the file that while ve validated.
        schema (Dict[str, Any]): the schema that the information needs to comply with.  
      
    Returns:  
        bool: `True` if the file complies with the schema, `False` otherwise.

    Raises:  
        jsonschema.exceptions.SchemaError: The configuration schema is invalid.
    """

    try:
        validate(file, schema)
    except jsonschema.exceptions.ValidationError as err:
        return False
    except jsonschema.exceptions.SchemaError as err:
        print(err)
        print("The Config_schema is invalid")
        raise err
    
    return True 

In [ ]:
#exporti
def open_config_file(config_path: Path) -> Dict[str, Any]:
    """Opens a json if it exists and matches a specific configuration schema.

    Args:  
        config_path (Path): path to the session's `config.json`  

    Returns:  
        Dict[str, Any]: dictionary extracted from `config.json`.  

    Raises:  
        FileNotFoundError: in case `config_path` doesn't point to an existing file.
    """
    if (
        config_path.exists() and 
        validate_config_file(json.load(config_path.open()), Config_schema)
    ):
        with config_path.open('r') as cf:
            return json.load(cf)
    else:
        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), config_path)

## The Module's Exported Functions
In this module, the only function designed to be exported for use elsewhere is `load_configurations`. This function is used in the `ingest` module to open and load the project's `config.json` file.


In [ ]:
#export
def load_configurations(config_path: Path) -> Config_settings:
    """
    ---
    
    Look for the project's `config.json` file in `config_path`, verifies that it contains the proper data and then returns a `Config_settings` object that contains the data.

    ----    

    **Args:**  
        *`config_path (Path)`:* path to the session's `config.json`  

    **Returns:**  
        *`Config_settings`:* frozen `dataclass` containing the data extracted from `config.json`.  
    """
    config_dict = open_config_file(config_path)
    return Config_settings(
        config_dict['PORT_ADDRESS'],
        config_dict['PORT_NUMBER'],
        config_dict['DB_NAME'],
        config_dict['REPLAY_PATH']
    )

In [ ]:
show_doc(load_configurations, title_level=3)

<h3 id="load_configurations" class="doc_header"><code>load_configurations</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h3>

> <code>load_configurations</code>(**`config_path`**:`Path`)

Look for the project's `config.json` file in `config_path`, verifies that it contains the proper data and then returns a [`Config_settings`](/sc2trainingg/load_config.html#Config_settings) object that contains the data.

----    

**Args:**  
    *`config_path (Path)`:* path to the session's `config.json`  

**Returns:**  
    *[`Config_settings`](/sc2trainingg/load_config.html#Config_settings):* frozen `dataclass` containing the data extracted from `config.json`.  

#### Example
In the following code `load_configurations` is used to load the data from a `config.json` in the project's folder. 

In [ ]:
print(load_configurations(Path("config.json")))

Config_settings(port_address='localhost', port_number=27017, db_name='TEST_library', replay_path='C:\\Users\\david\\Documents\\phdcode\\sc2trainingg\\test')


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_load_config.ipynb.
Converted 02_ingest.ipynb.
Converted index.ipynb.
